In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,RobustScaler
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest,f_regression,mutual_info_regression
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.neighbors import KNeighborsRegressor

from sklearn import metrics

my_NIA=262603
np.random.seed(my_NIA)

First of all we are going to download the data neccesary for this project. It is already splitted into training and testin, therefore we assign each of them to a variable which contains each part.

Then we select the features that we want to work with and the target variable for both, training and testing.

In order to be able to validate the model while training it, we are also going to split this first training data into training and validation.

In [2]:
train = pd.read_pickle('/Users/ignacioalmodovarcardenas/Desktop/Advanced programming/hyper_parameter_tuning/traintestdata_pickle/trainst1ns16.pkl')
test = pd.read_pickle('/Users/ignacioalmodovarcardenas/Desktop/Advanced programming/hyper_parameter_tuning/traintestdata_pickle/testst1ns16.pkl')

train_target=train.loc[:,"energy"]
test_target=test.loc[:,"energy"]

train_train=train.iloc[:,0:300]
test_test=test.iloc[:,0:300]

#Split into train and train validation
train_split=train_train.iloc[:3650,:]
train_validation=train_train.iloc[3650:,:]

train_target_split=train_target.iloc[:3650]
train_target_validation=train_target.iloc[3650:]

## NA imputation

Once we have the different parts of the datasets ready we want to impute missing values in random places of the data. 

We are going to select a 10% of the columns at random. This columns will be the ones were we are going to impute NAs. To do that we create an index with 30 variables taking values from 0 to 300.

With the information in this columns we want to impute a 10% of all the obsservations given. Therefore we generate another random index with that amount of values. 

To finish the imputation we generate another index with the same dimension as the one with all the observations using only the features generated in the first case.

Then using a simple loop we impute each NA in the place generated at random. We repit this process for both test and training sets.

In [3]:
train_NA_train=train_train.copy()
test_NA_test=test_test.copy()

#Take 30 columns at random
index_column_NA=np.random.choice(300,30)

#Take the total number of data and calculate its 10%, this will be the columns where we impute NAs
nNAs_train=int(index_column_NA.shape[0]*train_NA_train.shape[0]*0.1)
nNAs_test=int(index_column_NA.shape[0]*test_NA_test.shape[0]*0.1)

#Create a random index for the observations. The dimension will be the number of na neccesary
index_rNA_train=np.random.choice(int(train_NA_train.shape[0]),nNAs_train)
index_rNA_test=np.random.choice(int(test_NA_test.shape[0]),nNAs_test)

#Create a random index for the columns taking values only in the ones selected in the first place with dimension the number of na neccesary
index_cNA_train=np.random.choice(index_column_NA,nNAs_train)
index_cNA_test=np.random.choice(index_column_NA,nNAs_test)

#Impute na 
for n in range(index_cNA_train.shape[0]):
    train_NA_train.iloc[index_rNA_train[n],index_cNA_train[n]]=np.nan

for n in range(index_cNA_test.shape[0]):
    test_NA_test.iloc[index_rNA_test[n],index_cNA_test[n]]=np.nan
    

## Imputation methods

Once we have the data with NAs we are going to compare different options to impute missing values and see which one is the best one.

We are using knn algorithm. Therefore, we need to scale the data. As many oprations are needed, we are going to use pipelines to do this comparaissons much faster.

In [4]:
#Split NA dataset into train and train validation
train_NA_split=train_NA_train.iloc[:3650,:]
train_NA_validation=train_NA_train.iloc[3650:,:]

imputerMean=SimpleImputer(strategy="mean")
imputerMedian=SimpleImputer(strategy="median")

scaler_MinMax=MinMaxScaler()
scaler_Robust=RobustScaler()

knn=KNeighborsRegressor()

clf=Pipeline([
    ("scale",scaler_MinMax),
    ("impute",imputerMean),
    ("knn",knn)
    ])

clf2=Pipeline([
    ("scale",scaler_Robust),
    ("impute",imputerMean),
    ("knn",knn)
    ])

clf3=Pipeline([
    ("scale",scaler_MinMax),
    ("impute",imputerMedian),
    ("knn",knn)
    ])

clf4=Pipeline([
    ("scale",scaler_Robust),
    ("impute",imputerMedian),
    ("knn",knn)
    ])

#1st Pipe Scale MinMax, impute mean  **BEST ONE**
clf.fit(train_NA_split,train_target_split)
train_validation_imp1=clf.predict(train_NA_validation)
print(metrics.mean_absolute_error(train_validation_imp1,train_target_validation))

#2nd Pipe Scale Robust, impute mean
clf2.fit(train_NA_split,train_target_split)
train_validation_imp2=clf2.predict(train_NA_validation)
print(metrics.mean_absolute_error(train_validation_imp2,train_target_validation))

#3rd Pipe Scale MinMax, impute median
clf3.fit(train_NA_split,train_target_split)
train_validation_imp3=clf3.predict(train_NA_validation)
print(metrics.mean_absolute_error(train_validation_imp3,train_target_validation))

#4th Pipe Scale Robust, impute median
clf4.fit(train_NA_split,train_target_split)
train_validation_imp4=clf4.predict(train_NA_validation)
print(metrics.mean_absolute_error(train_validation_imp4,train_target_validation))

2461042.1161643835
3338346.256438356
2482044.815342466
3317272.9808219178


As we can see, depending on which scale method we use, we obtain similar results. When it comes to the imputation method the differences obtained are not very large when using the mean or the median value.

Hence, we can say that withing a small difference, the best combination is the first one, where the missing values are imputed following the mean value and the scalation is given by the MinMax algorithm.

## Feature selection

In this step we are going to proceed with feature selection. We are going to use two different methods. In the first case we are going to use SelectKbest and later we will use PCA. In both of this different methods we are going to see how the dimension of the dataset can be reduced without loosing much key information for our predictive models.

In [5]:
#Hyp tunning for knn
selection=SelectKBest(f_regression)

pipe1=Pipeline([
    ("scale",scaler_MinMax),
    ("impute",imputerMean),
    ("select",selection),
    ("knn",knn)
    ])

param_grid={
    "select__k":[int(x) for x in np.linspace(start = 1, stop = 300, num = 50)],
    "knn__n_neighbors":[2,4,8,16,32]
    }

train_cv_index=np.zeros(train_NA_train.shape[0]) 
train_cv_index[:3650] = -1
train_cv_index = PredefinedSplit(train_cv_index)

grid_tunning=GridSearchCV(pipe1,param_grid,
                          scoring="neg_mean_squared_error",cv=train_cv_index,n_jobs=-1,verbose=1)

grid_tunning.fit(train_NA_train,train_target)
print(grid_tunning.best_params_)
bestparams=grid_tunning.best_params_

pipe1.set_params(**bestparams)
pipe1.fit(train_NA_train,train_target)

model_select=pipe1.predict(train_NA_validation)
print(metrics.mean_absolute_error(model_select,train_target_validation))



Fitting 1 folds for each of 250 candidates, totalling 250 fits
{'knn__n_neighbors': 16, 'select__k': 269}
2088806.3393835616


As we can see, the best model in this case contains 269 columns and is evaluated using k=16. Which is indeed a very large number of columms. Therefore, the dimension compared to the original one have not been reduced a lot.

In [6]:
#PCA
pca=PCA()

pipe2=Pipeline([
    ("scale",scaler_MinMax),
    ("impute",imputerMean),
    ("pca",pca),
    ("knn",knn)
    ])

param_grid2={
    "pca__n_components":[int(x) for x in np.linspace(start = 1, stop = 300, num = 50)],
    "knn__n_neighbors":[2,4,8,16,32]
    }


grid_tunning2=GridSearchCV(pipe2,param_grid2,
                          scoring="neg_mean_squared_error",cv=train_cv_index,n_jobs=-1,verbose=1)

grid_tunning2.fit(train_NA_train,train_target)
print(grid_tunning2.best_params_)

model_PCA=grid_tunning2.predict(train_NA_validation)
print(metrics.mean_absolute_error(model_PCA,train_target_validation))

Fitting 1 folds for each of 250 candidates, totalling 250 fits
{'knn__n_neighbors': 32, 'pca__n_components': 13}
2160931.23390411


For this case, we can see that the optimal model uses the first 13 principal components and the number of neighbors k=32, which is twice the number used in the first method.

In constrast with the dimension obtained in the first method, using PCA we have significantly reduced it.

The result obtained is sligtly worse than the one without PCA. However, it is impresive considering that this model uses much less information.

## Extract names of attributes selected

In order to see which attributes seems to be more important for the imputation method we can get the scores for each one of them.

In [7]:
a=pipe1["select"]
columns_bool=a.get_support()
a.scores_

train_NA_train.columns[columns_bool].shape

col_importance=pd.DataFrame({"Columns":train_NA_train.columns[columns_bool],
                             "Scores":a.scores_[columns_bool]})

col_importance.sort_values(by="Scores",ascending=False)

,Columns,Scores
11,dswrf_s5_1,14430.933955
214,dswrf_s5_4,13735.971184
9,dswrf_s3_1,12958.454520
10,dswrf_s4_1,12878.727974
142,dswrf_s3_3,12809.606185
...,...,...
146,pres_ms4_3,262.235729
13,pwat_ea1_1,254.199299
145,pres_ms3_3,250.117787
68,apcp_sf2_2,249.846276


As we can see in the output, the first columns with higher scores are the ones corresponding to the Downward surface shortwave Flux, refers to the radioactive energy in the wavelength interval 0.3 $\mu m$, 4.0 $\mu m$.

In contrast the lower scores are given to variables like pwat and apcp, which stands for the  amount of water potentially available in the atmosphere for precipitation and 3-hour accumulated precipitation respectively.

## Evaluate best model

Using the scores obtained on the validation data for the PCA model and the one using SelectKBest, we got that the best one was the SelectKBest. Eventhough the difference was not very large, the imputation with the optimal number of column gave better results than the PCA one also optimized. 

In [8]:
model1=grid_tunning.predict(test_NA_test)
print(metrics.mean_absolute_error(model1,test_target))

2161832.374829468


The best result obtained in the previous task was 2312173. We used the knn method optimazed with k=7 which was the higher value in the interval given to do the hyperparameter tunning. 

In this scenario we can see that we have obtained better results using another knn model, in this case with k=16 and selecting only the most important features of the data. 

From a logic point of view, the error obtained in this model should be higher than the one used in the first task due to the information lost while for using the selectKbest() function. Nevertheless, as it can bee seen, this  is not the case and we have obtained better results using this model.

The fact that this method behavies better could be given by the fact that we are now only using k=16 for the knn algorithm or that the columns from the dataset that we are not considering in the model give information that penalizes the model instead of helping them to get better predictions. 
